## Existence Of Node Clusters

Here we demonstrate that in random forest that has been trained on some set of data, the nodes can be reasonably organized into clusters.

First, we must train or load a forest:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import sys
sys.path.append('../src/python')
import tree_reader as tr 
import lumberjack

data_location = "/Users/bbrener1/battle/rusty_forest_4/data/aging_brain/"
!ls {data_location}
# data_location = "../data/aging_brain/"

forest = tr.Forest.load(data_location + 'cv_forest_trimmed_extra')
forest.arguments


GSE129788_RAW.tar                       cv_forest_trimmed
GSM3722100_YX1L_10X.txt                 cv_forest_trimmed_extra
GSM3722101_YX2L_10X.txt                 cv_forest_trimmed_extra_1
GSM3722102_YX3R_10X.txt                 cv_forest_trimmed_extra_10
GSM3722103_YX4R_10X.txt                 cv_forest_trimmed_extra_2
GSM3722104_YX5R_10X.txt                 cv_forest_trimmed_extra_3
GSM3722105_YX6L_10X.txt                 cv_forest_trimmed_extra_4
GSM3722106_YX7R_10X.txt                 cv_forest_trimmed_extra_5
GSM3722107_YX8L_10X.txt                 cv_forest_trimmed_extra_5_cached
GSM3722108_OX1X_10X.txt                 cv_forest_trimmed_extra_5_cached_great
GSM3722109_OX2X_10X.txt                 cv_forest_trimmed_extra_5_cachked_great
GSM3722110_OX3X_10X.txt                 cv_forest_trimmed_extra_6
GSM3722111_OX4X_10X.txt                 cv_forest_trimmed_extra_7
GSM3722112_OX5X_10X.txt                 cv_forest_trimmed_extra_8
GSM3722113_OX6X_10X.txt                 cv_forest_t

['/Users/bbrener1/battle/rusty_forest_4/target/release/rusty_lumberjack_v3',
 '-ic',
 '/var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmp1p802p8o/input.counts',
 '-oc',
 '/var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmp1p802p8o/output.counts',
 '-o',
 '/var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmp1p802p8o/tmp',
 '-auto',
 '-ifh',
 '/var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmp1p802p8o/tmp.ifh',
 '-ofh',
 '/var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmp1p802p8o/tmp.ofh',
 '-trees',
 '100',
 '-braids',
 '2',
 '-ifs',
 '150',
 '-ofs',
 '150',
 '-ss',
 '500',
 '-depth',
 '8',
 '-leaves',
 '10',
 '-sfr',
 '0',
 '-norm',
 'l1',
 '-reduce_input',
 'true',
 '-reduce_output',
 'false']

In [2]:

print(len(forest.output_features))
print(len(forest.split_clusters))
print(len(forest.nodes()))



351
39
14082


A Random Forest is a collection of decision trees, and a decision tree is a collection of individual decision points, commonly known as "Nodes"

To understand Random Forests and Decision Trees, it is important to understand how Nodes work. Each individual node is a (very crappy) regressor, eg. each Node makess a prediction based on a rule like "If Gene 1 has expression > 10, Gene 2 will have expression < 5", or "If a house is < 5 miles from a school, it will cost > $100,000". A very important property of each node, however, is that it can also have children, which are other nodes. When a node makes a prediction like "If Gene 1 has expression > 10 then Gene 2 has expression < 5", it can pass all the samples for which Gene 1 is > 10 to one of its children, and all the samples for which Gene 1 < 10 to the other child. After that, each one of its children can make a different prediction, which results in compound rules.

This is how a decision tree is formed. A decision tree with a depth of 2 might contain a rule like "If Gene 1 > 10 AND Gene 3 > 10, THEN Gene 2 and Gene 4 are both < 2, which would represent one of the "Leaf" nodes that it has. Leaf nodes are nodes with no children. 

Individual decision trees, then, are somewhat crappy predictors, but they're better than individual nodes. In order to improve the performance of decision trees, we can construct a Random Forest. To construct a random forest, we can train many decision trees on bootstraps of a dataset

If many decision trees are combined and their predictions averaged together, you have a Random Forest, which is a pretty good kind of regressor. 

A practical demonstration might help:

In [ ]:
forest.reset_split_clusters()
forest.interpret_splits(depth=5,mode='additive_mean',metric='cosine',pca=100,relatives=True,k=10,resolution=1)

So now that we know that random forests are collections of ordered nodes, we can examine a more interesting question: do certain nodes occur repeatedly in the forest, despite operating on bootstrapped samples? 

In order to examine this question first we must understand different ways of describing a node. I think generally there are three helpful ways of looking at a node:

* **Node Sample Encoding**: A binary vector the length of the number of samples you are considering. 0 or false means the sample is absent from the node. A 1 or true means the sample is present in the node. 

* **Node Mean Encoding**: A float vector the length of the number of targets you are considering. Each value is the mean of the target values for all samples in this node. This is the node's prediction for samples that occur in it.

* **Node Additive Encoding**: A float vector the length of the number of targets you are considering. Each value is THE DIFFERENCE between the mean value for that target in THIS NODE and the mean value for that target IN THE PARENT of this node. For root nodes, which have no parents, the additive encoding is simply th mean value across the entire dataset. (As if the mean of a hypothetical parent would have been 0). This encoding represents the marginal effect of each node.

We should examine if there are any common patterns that appear if we encode many nodes from a forest using each of these representations:

In [ ]:
# Here we plot the sample representations of nodes. 
# This generates a set of figures demonstrating the existence of node clusters

from sklearn.decomposition import PCA

# For ease of processing we have to construct dimensionally reduced representations of the encodings. 

# sample_encoding = forest.node_representation(forest.nodes(root=False),mode='sample')
# reduced_sample = PCA(n_components=100).fit_transform(sample_encoding.T)
# reduced_node = PCA(n_components=100).fit_transform(sample_encoding)

# print(sample_encoding.shape)
# print(reduced_sample.shape)
# print(reduced_node.shape)

from scipy.cluster.hierarchy import linkage,dendrogram

# sample_agglomeration = dendrogram(linkage(reduced_sample, metric='cosine', method='average'), no_plot=True)['leaves']
# node_agglomeration = dendrogram(linkage(reduced_node, metric='cosine', method='average'), no_plot=True)['leaves']

# plt.figure()
# plt.title("Cell Presence in Node (Two-Way Agglomerated)")
# plt.imshow(sample_encoding[node_agglomeration].T[sample_agglomeration].T,cmap='binary',aspect='auto',interpolation='none')
# plt.xlabel("Cells")
# plt.ylabel("Nodes")
# plt.colorbar()
# plt.tight_layout()
# plt.show()

# # And here we sort the nodes after they have been clustered (more on the clustering procedure in a bit)

# node_cluster_sort = np.argsort([n.split_cluster for n in forest.nodes(root=False)])

# plt.figure()
# plt.title("Cell Presence in Node (Clustered)")
# plt.imshow(sample_encoding[node_cluster_sort].T[sample_agglomeration].T,cmap='binary',aspect='auto',interpolation='none')
# plt.xlabel("Cells")
# plt.ylabel("Nodes")
# plt.colorbar()
# plt.tight_layout()
# plt.show()


In [ ]:
## from sklearn.decomposition import PCA

sister_encoding = forest.node_representation(forest.nodes(root=False),mode='sister')
reduced_sister = PCA(n_components=100).fit_transform(sister_encoding.T)
reduced_node = PCA(n_components=100).fit_transform(sister_encoding)

print(sister_encoding.shape)
print(reduced_sister.shape)
print(reduced_node.shape)

# from scipy.cluster.hierarchy import linkage,dendrogram

# sister_agglomeration = dendrogram(linkage(reduced_sister, metric='cosine', method='average'), no_plot=True)['leaves']
# node_agglomeration = dendrogram(linkage(reduced_node, metric='cosine', method='average'), no_plot=True)['leaves']

# plt.figure()
# plt.title("Sample Presence in Node vs Sister (Two-Way Agglomerated)")
# plt.imshow(sister_encoding[node_agglomeration].T[sister_agglomeration].T,cmap='bwr',aspect='auto',interpolation='none')
# plt.xlabel("Samples")
# plt.ylabel("Nodes")
# plt.colorbar()
# plt.tight_layout()
# plt.show()

# plt.figure()
# plt.title("Sample Presence in Node vs Sister (Clustered By Gain)")
# plt.imshow(sister_encoding[node_cluster_sort].T[sister_agglomeration].T,cmap='bwr',aspect='auto',interpolation='none')
# plt.xlabel("Samples")
# plt.ylabel("Nodes")
# plt.colorbar()
# plt.tight_layout()
# plt.show()


In [ ]:
# Here we plot the construct and agglomerate the additive gain representation 


feature_encoding = forest.node_representation(forest.nodes(root=False),mode='partial')
reduced_feature = PCA(n_components=100).fit_transform(feature_encoding.T)
reduced_node = PCA(n_components=100).fit_transform(feature_encoding)

feature_agglomeration = dendrogram(linkage(reduced_feature, metric='cosine', method='average'), no_plot=True)['leaves']
node_agglomeration = dendrogram(linkage(reduced_node, metric='cosine', method='average'), no_plot=True)['leaves']

node_cluster_sort = np.argsort([n.split_cluster for n in forest.nodes(root=False)])

In [ ]:
# Here we plot the additive gain representation 

print(feature_encoding.shape)

plt.figure()
plt.title("Target Gain in Node (Double-Agglomerated)")
plt.imshow(feature_encoding[node_agglomeration].T[feature_agglomeration].T,cmap='bwr',interpolation='none',aspect='auto',vmin=-2,vmax=2)
plt.xlabel("Genes")
plt.ylabel("Nodes")
plt.colorbar(label="Parent Mean - Node Mean (Log TPM)")
plt.tight_layout()
plt.show()

plt.figure()
plt.title("Target Gain in Node (Clustered)")
plt.imshow(feature_encoding[node_cluster_sort].T[feature_agglomeration].T,cmap='bwr',interpolation='none',aspect='auto',vmin=-2,vmax=2)
plt.xlabel("Genes")
plt.ylabel("Nodes")
plt.colorbar(label="Parent Mean - Node Mean (Log TPM)")
plt.tight_layout()
plt.show()

Finally we can look at silhouette plots scores for various node encodings in order to get a feel for whether or not we are adequately clustering them and whether or not the clusters meaningfully exist. 

In [ ]:
# Silhouette Plots For Node Clusters 

from sklearn.metrics import silhouette_samples, silhouette_score

node_labels = np.array([n.split_cluster for n in forest.nodes(root=False)])

# silhouette_scores = silhouette_samples(reduced_node,node_labels,metric='cosine')
silhouette_scores = silhouette_samples(feature_encoding,node_labels,metric='euclidean')
# silhouette_scores = silhouette_samples(sample_encoding,node_labels,metric='cosine')
# silhouette_scores = silhouette_samples(sister_encoding,node_labels,metric='cosine')

sorted_silhouette = np.zeros(silhouette_scores.shape)
sorted_colors = np.zeros(silhouette_scores.shape)

current_index = 0
next_index = 0
for i in sorted(set(node_labels)):
    mask = node_labels == i
    selected_values = sorted(silhouette_scores[mask])    
    next_index = current_index + np.sum(mask)
    sorted_silhouette[current_index:next_index] = selected_values
    sorted_colors[current_index:next_index] = i
    current_index = next_index

In [ ]:
import matplotlib.cm as cm

plt.figure()
plt.title("Silhouette Plots For Nodes Clustered By Gain")
for i,node in enumerate(sorted_silhouette):
    plt.plot([0,node],[i,i],color=cm.nipy_spectral(sorted_colors[i] / len(forest.split_clusters)),linewidth=0.5)
# plt.scatter(sorted_silhouette,np.arange(len(sorted_silhouette)),s=1)
plt.plot([0,0],[0,len(sorted_silhouette)],color='red')
plt.xlabel("Silhouette Score")
plt.ylabel("Nodes")
plt.show()

In [ ]:

node_populations = np.array([n.pop() for n in forest.nodes(root=False)])
mask = node_populations > 100


feature_encoding = forest.node_representation(forest.nodes(root=False),mode='partial')[mask]
reduced_feature = PCA(n_components=100).fit_transform(feature_encoding.T)
reduced_node = PCA(n_components=100).fit_transform(feature_encoding)


feature_agglomeration = dendrogram(linkage(reduced_feature, metric='cosine', method='average'), no_plot=True)['leaves']
node_agglomeration = dendrogram(linkage(reduced_node, metric='cosine', method='average'), no_plot=True)['leaves']

node_cluster_sort = np.argsort(np.array([n.split_cluster for n in forest.nodes(root=False)])[mask])

In [ ]:

print(feature_encoding.shape)

plt.figure()
plt.title("Target Gain in Node (Double-Agglomerated)")
plt.imshow(feature_encoding[node_agglomeration].T[feature_agglomeration].T,cmap='bwr',interpolation='none',aspect='auto',vmin=-2,vmax=2)
plt.xlabel("Genes")
plt.ylabel("Nodes")
plt.colorbar(label="Parent Mean - Node Mean (Log TPM)")
plt.tight_layout()
plt.show()


plt.figure()
plt.title("Target Gain in Node (Clustered)")
plt.imshow(feature_encoding[node_cluster_sort].T[feature_agglomeration].T,cmap='bwr',interpolation='none',aspect='auto',vmin=-2,vmax=2)
plt.xlabel("Genes")
plt.ylabel("Nodes")
plt.colorbar(label="Parent Mean - Node Mean (Log TPM)")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score

node_labels = np.array([n.split_cluster for n in forest.nodes(root=False)])[mask]

# silhouette_scores = silhouette_samples(reduced_node,node_labels,metric='cosine')
silhouette_scores = silhouette_samples(feature_encoding,node_labels,metric='euclidean')
# silhouette_scores = silhouette_samples(sample_encoding,node_labels,metric='cosine')
# silhouette_scores = silhouette_samples(sister_encoding,node_labels,metric='cosine')

sorted_silhouette = np.zeros(silhouette_scores.shape)
sorted_colors = np.zeros(silhouette_scores.shape)

current_index = 0
next_index = 0
for i in sorted(set(node_labels)):
    mask = node_labels == i
    selected_values = sorted(silhouette_scores[mask])    
    next_index = current_index + np.sum(mask)
    sorted_silhouette[current_index:next_index] = selected_values
    sorted_colors[current_index:next_index] = i
    current_index = next_index

In [ ]:
import matplotlib.cm as cm

plt.figure()
plt.title("Silhouette Plots For Nodes Clustered By Gain")
for i,node in enumerate(sorted_silhouette):
    plt.plot([0,node],[i,i],color=cm.nipy_spectral(sorted_colors[i] / len(forest.split_clusters)),linewidth=0.5)
# plt.scatter(sorted_silhouette,np.arange(len(sorted_silhouette)),s=1)
plt.plot([0,0],[0,len(sorted_silhouette)],color='red')
plt.xlabel("Silhouette Score")
plt.ylabel("Nodes")
plt.show()

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
reduced_feature.shape

In [ ]:
trans_node = TSNE(n_components=2,metric='cosine').fit_transform(reduced_node)

In [ ]:
plt.figure()
plt.imshow(reduced_node[node_agglomeration][:,:20],aspect='auto',cmap='bwr',vmin=-20,vmax=20,interpolation='none')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.scatter(*trans_node.T,s=2,c=node_labels,cmap='rainbow')
plt.show()

In [ ]:
from sklearn.cluster import KMeans

for i in range(5,50,5):

    node_labels = KMeans(i).fit_predict(reduced_node[:,:20])
        
    silhouette_scores = silhouette_samples(feature_encoding,node_labels,metric='cosine')

    sorted_silhouette = np.zeros(silhouette_scores.shape)
    sorted_colors = np.zeros(silhouette_scores.shape)

    current_index = 0
    next_index = 0
    for i in sorted(set(node_labels)):
        mask = node_labels == i
        selected_values = sorted(silhouette_scores[mask])    
        next_index = current_index + np.sum(mask)
        sorted_silhouette[current_index:next_index] = selected_values
        sorted_colors[current_index:next_index] = i
        current_index = next_index
        
        
    plt.figure()
    plt.title("Silhouette Plots For Nodes Clustered By Gain")
    for i,node in enumerate(sorted_silhouette):
        plt.plot([0,node],[i,i],color=cm.nipy_spectral(sorted_colors[i] / len(forest.split_clusters)),linewidth=0.5)
    # plt.scatter(sorted_silhouette,np.arange(len(sorted_silhouette)),s=1)
    plt.plot([0,0],[0,len(sorted_silhouette)],color='red')
    plt.xlabel("Silhouette Score")
    plt.ylabel("Nodes")
    plt.show()

In [ ]:
len(set(clustered))

In [ ]:
len(clustered)

In [ ]:
optics_sort = np.argsort(clustered)

plt.figure()
plt.imshow(reduced_node[optics_sort][:,:20],aspect='auto',cmap='bwr',vmin=-20,vmax=20,interpolation='none')
plt.colorbar()
plt.show()

In [20]:
feature_encoding = forest.node_representation(forest.nodes(root=False),mode='additive_mean')
labels = np.array([n.split_cluster for n in forest.nodes(root=False)])

Additive mean reduction


In [21]:
feature_encoding.shape


(13982, 351)

In [14]:
remaining = 0

for cluster in sorted(list(set(labels))):
    mask = labels == cluster
    means = np.mean(feature_encoding[mask],axis=0)
    residuals = feature_encoding[mask] - means
    mse = np.sum(np.power(residuals,2)) / (np.sum(mask) * feature_encoding.shape[1])
    remaining += (np.sum(mask) / feature_encoding.shape[0]) * mse
    print((cluster,mse))
    
means = np.mean(feature_encoding,axis=0)
residuals = feature_encoding - means
mse = np.sum(np.power(residuals,2)) / (feature_encoding.shape[0] * feature_encoding.shape[1])
print(f"Remaining: {remaining}")
print(f"All:{mse}")


(1, 0.016559641797076505)
(2, 0.026719961011696415)
(3, 0.06821156766792495)
(4, 0.010066074929283925)
(5, 0.09240248073391005)
(6, 0.025061590307325306)
(7, 0.02991602035615037)
(8, 0.0353980649232254)
(9, 0.03799498937860699)
(10, 0.010433189810988633)
(11, 0.04613366957511943)
(12, 0.036096933258465906)
(13, 0.018153790308255306)
(14, 0.09872839314956372)
(15, 0.028664263348692515)
(16, 0.03513532089474127)
(17, 0.02862872277066524)
(18, 0.05273124479343409)
(19, 0.07982630344261693)
(20, 0.04620545624633957)
(21, 0.016000652388528186)
(22, 0.027374834552137576)
(23, 0.005378702338809322)
(24, 0.05956778562472024)
(25, 0.070170598958087)
(26, 0.03685004444040585)
(27, 0.038480988056391524)
(28, 0.007310224725935504)
(29, 0.011195267947414662)
(30, 0.009261578022057989)
(31, 0.037641602599044345)
(32, 0.04525645099510132)
(33, 0.0263833937966067)
(34, 0.01577973941522759)
(35, 0.05288525227299987)
(36, 0.02105744470411041)
(37, 0.04311471398790514)
(38, 0.010435361956718195)
Remainin

In [23]:
from tree_reader_utils import fast_knn,hacked_louvain

shuffled = feature_encoding.copy()

for f in shuffled.T:
    np.random.shuffle(f)



In [28]:
relabel = hacked_louvain(fast_knn(shuffled,50))
    

In [31]:
remaining_sims = []

for i in range(20):
    print(i)
    
    shuffled = feature_encoding.copy()

    for f in shuffled.T:
        np.random.shuffle(f)

    relabel = hacked_louvain(fast_knn(shuffled,50))

    remaining = 0
    for cluster in sorted(list(set(relabel))):
        mask = relabel == cluster
        means = np.mean(shuffled[mask],axis=0)
        residuals = shuffled[mask] - means
        mse = np.sum(np.power(residuals,2)) / (np.sum(mask) * shuffled.shape[1])
        remaining += (np.sum(mask) / shuffled.shape[0]) * mse
#         print((cluster,mse))
    remaining_sims.append(remaining)
    
means = np.mean(shuffled,axis=0)
residuals = shuffled - means
mse = np.sum(np.power(residuals,2)) / (shuffled.shape[0] * shuffled.shape[1])
print(f"Remaining: {remaining}")
print(f"All:{mse}")


0


Searching for partition
Louvain: (13982,)
1


Searching for partition
Louvain: (13982,)
2


Searching for partition
Louvain: (13982,)
3


Searching for partition
Louvain: (13982,)
4


Searching for partition
Louvain: (13982,)
5


Searching for partition
Louvain: (13982,)
6


Searching for partition
Louvain: (13982,)
7


Searching for partition
Louvain: (13982,)
8


Searching for partition
Louvain: (13982,)
9


Searching for partition
Louvain: (13982,)
10


Searching for partition
Louvain: (13982,)
11


Searching for partition
Louvain: (13982,)
12


KeyboardInterrupt: 

In [39]:
np.var(remaining_sims)
# np.mean(remaining_sims)

2.069990830860928e-08

In [36]:
remaining_sims

[0.05971537755354919,
 0.05961775082241294,
 0.059816129397441464,
 0.05987840564796268,
 0.05987251926929091,
 0.05943743881797868,
 0.059777988404586146,
 0.06001173201954825,
 0.05975851649985739,
 0.059672617866716224,
 0.05976588750782693,
 0.05990828552112961]

In [37]:
mse

0.06315214326262976